In [1]:
import pandas as pd
from pulp import *

# Manufacturing Variable Costs 

In [2]:
# Import Costs
mvc = pd.read_excel("/kaggle/input/supplycchain/variable_costs.xlsx", index_col = 0)
print(mvc)

                         USA  Germany  Japan  Brazil  India
Variable Costs ($/Unit)                                    
USA                       12       12     12      12     12
Germany                   13       13     13      13     13
Japan                     10       10     10      10     10
Brazil                     8        8      8       8      8
India                      5        5      5       5      5


#  Freight Costs

In [3]:
# Import Costs
fc = pd.read_excel("/kaggle/input/supplycchain/freight_costs.xlsx", index_col = 0)
print(fc)

                               USA  Germany  Japan  Brazil  India
Freight Costs ($/Container)                                      
USA                              0    12250   1100   16100   8778
Germany                      13335        0   8617   20244  10073
Japan                        15400    22750      0   43610  14350
Brazil                       16450    22050  28000       0  29750
India                        13650    15400  24500   29400      0


# Variable Costs

In [4]:
# Variable Costs
variable_cost = fc/1000 + mvc

print(variable_cost)

                                USA  Germany   Japan  Brazil   India
Freight Costs ($/Container)                                         
USA                          12.000    24.25  13.100  28.100  20.778
Germany                      26.335    13.00  21.617  33.244  23.073
Japan                        25.400    32.75  10.000  53.610  24.350
Brazil                       24.450    30.05  36.000   8.000  37.750
India                        18.650    20.40  29.500  34.400   5.000


# Fixed Costs

In [5]:
# Import Costs
fixed_costs = pd.read_excel("/kaggle/input/supplycchain/fixed_cost.xlsx", index_col = 0)
print(fixed_costs)

          Low  High
USA      6500  9500
Germany  4980  7270
Japan    6230  9100
Brazil   3230  4730
India    2110  6160


# Plants Capacity

In [6]:
# Two types of plants: Low Capacity and High Capacity Plant
cap = pd.read_excel("/kaggle/input/supplycchain/capacity.xlsx", index_col = 0)
print(cap)

                         Low  High
Capacity (kUnits/month)           
USA                      500  1500
Germany                  500  1500
Japan                    500  1500
Brazil                   500  1500
India                    500  3000


# Demand

In [7]:
demand = pd.read_excel("/kaggle/input/supplycchain/demand_1.xlsx", index_col= 0)
print(demand)

                Demand
(Units/month)         
USA            2800000
Germany          90000
Japan          1700000
Brazil          145000
India           160000


# Define Decision Variables

In [8]:
# Define Decision Variables
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low', 'High']

# Initialize Class
model = LpProblem("Capacitated Plant Location Model", LpMinimize)


# Create Decision Variables
x = LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='continuous')
y = LpVariable.dicts("plant_", 
                     [(i,s) for s in size for i in loc], cat='Binary')

# Define Objective Function
model += (lpSum([fixed_costs.loc[i,s] * y[(i,s)] * 1000 for s in size for i in loc])
          + lpSum([variable_cost.loc[i,j] * x[(i,j)]   for i in loc for j in loc]))

# Add Constraints
for j in loc:
    model += lpSum([x[(i, j)] for i in loc]) == demand.loc[j,'Demand']
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s]*y[(i,s)] * 1000
                                                       for s in size])


# Define logical constraint: Add a logical constraint so that if the high capacity plant in USA is open, then a low capacity plant in Germany is also opened.
# model += y[('USA','High_Cap')] <= y[('Germany','Low_Cap')]                                                       
                                                       
# Solve Model
model.solve()
print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))


# Dictionnary
dict_plant = {}
dict_prod = {}
for v in model.variables():
    if 'plant' in v.name:
        name = v.name.replace('plant__', '').replace('_', '')
        dict_plant[name] = int(v.varValue)
        p_name = name
    else:
        name = v.name.replace('production__', '').replace('_', '')
        dict_prod[name] = v.varValue
    print(name, "=", v.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/07982c86824b4dd7b81d7e555482a4d2-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/07982c86824b4dd7b81d7e555482a4d2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 131 RHS
At line 142 BOUNDS
At line 153 ENDATA
Problem MODEL has 10 rows, 35 columns and 60 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 8.7227e+07 - 0.00 seconds
Cgl0004I processed model has 10 rows, 35 columns (10 integer (10 of which binary)) and 60 elements
Cbc0038I Initial state - 3 integers unsatisfied sum - 0.603333
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 9.3579e+07 iterations 5
Cbc0038I Solution found of 9.3579e+07
Cbc0038I Relaxing continuous gives 9.3579e+07
Cbc0038I Before mini branch and bound,

/opt/conda/lib/python3.10/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


# Capacity Plant

In [9]:
# Capacity Plant
list_low, list_high = [], []
for l in loc:
    for cap in ['Low', 'High']:
        x = "('{}','{}')".format(l, cap)
        if cap == 'Low':
            list_low.append(dict_plant[x])
        else:
            list_high.append(dict_plant[x])
df_capacity = pd.DataFrame({'Location': loc, 'Low': list_low, 'High': list_high}).set_index('Location')
    
df_capacity

,Low,High
Location,,
USA,0,1
Germany,0,0
Japan,0,1
Brazil,1,0
India,0,1
